<a href="https://colab.research.google.com/github/Anjali-K-S25/AI-Powered-health-assistant/blob/main/app_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers torch accelerate gradio

In [ ]:
!pip install bitsandbytes
import bitsandbytes
print("✅ bitsandbytes is installed correctly!")

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

print("⏳ Loading BioMistral-7B model in 8-bit mode...")

bnb_config = BitsAndBytesConfig(load_in_8bit=True)

tokenizer = AutoTokenizer.from_pretrained("BioMistral/BioMistral-7B")
model = AutoModelForCausalLM.from_pretrained(
    "BioMistral/BioMistral-7B",
    quantization_config=bnb_config,  # Enable 8-bit mode
    device_map="auto"
)

print("✅ Model loaded successfully in 8-bit mode!")

In [ ]:
import torch
import gradio as gr

# ✅ Define Chatbot Function
def chat_with_meditron(user_query):
    if not user_query.strip():
        return "⚠️ Please enter a valid medical question."

    try:
        device = "cuda" if torch.cuda.is_available() else "cpu"
        inputs = tokenizer(user_query, return_tensors="pt").to(device)

        # **Optimized generation settings**
        output = model.generate(
            **inputs,
            max_new_tokens=200,  # Restrict response length
            temperature=0.5,  # More focused responses
            top_p=0.8,  # Reduce hallucinations
            repetition_penalty=1.2  # Avoid repeating phrases
        )

        response = tokenizer.decode(output[0], skip_special_tokens=True)

        # **Filter out irrelevant answers**
        if "I'm sorry" in response or len(response.split()) < 5:
            return "⚠️ I couldn't find a relevant medical answer. Please rephrase your question."

        return response

    except Exception as e:
        return f"⚠️ Error: {str(e)}"

# ✅ Define Gradio UI
with gr.Blocks(theme=gr.themes.Soft()) as app:
    with gr.Tab("🏥 Main Page"):
        gr.Markdown("<h1 style='text-align: center;'>🏥 AI Health Assistant</h1>")
        gr.Markdown("<p style='text-align: center; font-size:18px;'>Ask any medical-related question and get AI-generated insights.</p>")

        user_input = gr.Textbox(label="Enter your medical query:")
        submit_button = gr.Button("🔍 Get Answer", variant="primary")
        chatbot_response = gr.Textbox(label="Chatbot Response", interactive=False)

        submit_button.click(chat_with_meditron, inputs=user_input, outputs=chatbot_response)

    with gr.Tab("ℹ️ About"):
        gr.Markdown("<h2>ℹ️ About the AI Health Assistant</h2>")
        gr.Markdown("- 🏥 This AI chatbot answers **medical-related questions**.\n- ⚠️ **Not a substitute for professional medical advice**.\n- 🚀 Built using **Gradio** and **Hugging Face Models**.")

    with gr.Tab("❓ FAQ"):
        gr.Markdown("<h2>❓ Frequently Asked Questions</h2>")

        gr.Markdown("**1️⃣ Can I use this for medical diagnosis?**\n- ❌ No, this is for **informational purposes only**. Always consult a healthcare professional for medical concerns.")

        gr.Markdown("**2️⃣ How accurate are the responses?**\n- 📊 The AI provides answers based on trained medical data, but accuracy depends on the complexity of the question. Cross-check with reliable sources.")

        gr.Markdown("**3️⃣ Is my data safe?**\n- 🔐 Yes, your input is **not stored**. This chatbot runs locally or through a secure cloud instance.")

        gr.Markdown("**4️⃣ What types of medical questions can I ask?**\n- 💡 You can ask about **symptoms, diseases, medications, treatments, and general health advice**. However, it does **not replace professional medical consultation**.")

        gr.Markdown("**5️⃣ Does this AI prescribe medicine?**\n- ❌ No, this chatbot does **not prescribe medications**. Always consult a doctor before taking any medicine.")

        gr.Markdown("**6️⃣ Can it provide emergency medical advice?**\n- ⚠️ No, this AI is **not for emergency situations**. If you have a medical emergency, please call **emergency services immediately**.")

        gr.Markdown("**7️⃣ Is the chatbot suitable for mental health support?**\n- 🧠 While it can provide **basic mental health guidance**, it is **not a substitute for professional therapy**. If you're struggling, seek help from a mental health professional.")

        gr.Markdown("**8️⃣ Can I trust the medical advice given?**\n- 🏥 The AI provides information based on medical sources, but it should **not be your only source**. Always verify with medical professionals or trusted resources like the **WHO or CDC**.")

        gr.Markdown("**9️⃣ How frequently is the AI updated?**\n- 🔄 The AI model is periodically updated based on available **medical research** and **AI improvements**. However, it's always best to **cross-check medical advice**.")

        gr.Markdown("**🔟 Can I use this chatbot for educational purposes?**\n- 📚 Yes! This chatbot is great for **learning about medical topics** but should not be used for clinical decision-making.")

# ✅ Launch the app
app.launch(share=True)